<a href="https://colab.research.google.com/github/shaunboodram/GROUP-9-PHAS0052-2022/blob/main/Make_SM_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import yoda


f = "/unix/cedar/xwang/contur/OpenLoops/sherpa/ATLAS_2019_I1764342.yoda"        # path to the yoda file
aos = yoda.read(f)
ao_2dScatter = []
for path, ao in aos.items():
    ao.setPath("/THY/"+path[1:])
    ao2d = ao.mkScatter()              # convert Histo1D to Scatter2D, then calculate weight contribution
    ao_2dScatter.append(ao2d)
yoda.write(ao_2dScatter,"ATLAS_2019_I1764342-Scatter2D.yoda")    # write scatter 2D yoda file

y_values = []
aoss = yoda.read("ATLAS_2019_I1764342-Scatter2D.yoda")  
for path, a in aoss.items():
    if type(a) == yoda.core.Scatter1D:           # scatter 1D object does not have y value
       break
    y_values.append(a.yVals())      
    with open("ATLAS_2019_I1764342-Scatter2D.yoda", "r+") as f:
        
        # empty lists for all/upper/lower errors for different d0x
        All = []     
        Upper = []    
        Lower = []     
        
        n_list = np.array([0,17,34,51,68,85])            # index of nominal values
        var_list = np.array([2,3,4,5,6,7,8,9,10,11,12,13,14,16])   # index of weights, exclude MEWeight and NTrial (MC event generator weights)
        for n in n_list:                 # path, d02 ~ d07
            for var in var_list+n:    # weights
                new_val = y_values[var]   # weighted values
                nominal_val = y_values[n]  # nominal value
                all_errors = (new_val - nominal_val)**2 #it was new_val - nominal_val for last test, need to check if the diff is between nominal and value or average and value
                
                if new_val >= nominal_val:
                    upperErrors = (new_val - nominal_val)**2
                else:
                    lowerErrors = (new_val - nominal_val)**2
                
                # all/upper/lower errors with specific d0x and all weights
                a.append(all_errors)     
                u.append(upperErrors)
                l.append(lowerErrors)
            # put all/upper/lower errors with different d0x and variation to the list
            all += [a]                  
            upper += [u]
            lower += [l]
            # number of all/upper/lower errors, exclude 0 terms
            All_len.append(len(list(filter((0).__ne__,a))))
            Upper_len.append(len(list(filter((0).__ne__,u))))
            Lower_len.append(len(list(filter((0).__ne__,l))))
            # sum all the varition contribution for each d0x
            All = np.sum(all[path.index('d')])
            Upper = np.sum(upper[path.index('d')])
            Lower = np.sum(lower[path.index('d')])
            # clear all/upper/lower errors lists with specific d0x to run a new list of the next d0x 
            a = []                                          
            u = []                   
            l = []
        
            if (np.all(Upper[path.index('d')])!=0):
                upperRMS = (abs(Upper[path.index('d')])/Upper_len[path.index('d')])**0.5
            else:
                upperRMS = 0.
            if (np.all(Lower[path.index('d')])!=0):
                lowerRMS = (abs(L[path.index('d')])/Llen[path.index('d')])**0.5
            else:
                lowerRMS = 0.
            if (A[path.index('d')]!=0):
                allRMS = (abs(A[path.index('d')])/Alen[path.index('d')])**0.5
                print(A[path.index('d')])
            else:
                allRMS = 0.
            #### set yval and yerr+-
            a.setYErrs(upperRMS,lowerRMS)
            a.setTitle(prediction.short_description)
            anaObjects.append(a)
d = [anaObjects[0],anaObjects[17],anaObjects[34],anaObjects[51],anaObjects[68],anaObjects[85]]
yoda.write(anaObjects,"ATLAS_2019_I1764342-new.yoda")